In [197]:
import pandas as pd
import datetime
import re
import glob

In [198]:
path = r'C:\Users\Aidan\OneDrive - Simon Fraser University (1sfu)\Garbage Route Optimization\Weekly Weight and Tips' # use your path
all_files = glob.glob(path + "/*.xls")
li = []
for filename in all_files:
    df = pd.read_excel(filename,
                  header=[0,1])
    li.append(df)

In [199]:
for i in range(0,len(li)):
    li[i].columns=li[i].columns.to_flat_index()
    li[i] = li[i].iloc[1:,]

In [200]:
# Creating data frame

# This contains granular info
dailyInfo = pd.DataFrame(columns=['fileName',
                                  'date',
                                  'zone',
                                  'driver',
                                  'truckNumber',
                                  'garbageTips',
                                  'garbageWeight',
                                  'garbageTime',
                                 'recyclingTips',
                                 'recyclingWeight',
                                 'recyclingTime',
                                 'greenTips',
                                 'greenWeight',
                                 'greenTime'])

# Note I dont bother with the totals since they can be calculated from the above dataframe

In [ ]:
# Regulard expression to capture the zone
zoneExpression = re.compile(r'[Zz]one\s*\S*')
customerExpression = re.compile(r'[Cc]ustomer')


for i in range(0,len(li)):
    # Getting the df we are working with
    df = li[i]
    
    # Getting the filename for reference
    fileName = all_files[0][all_files[0].rfind('\\')+1:]
    
    # Zeroing these out, these need to be non to detect when we need to look ahead to find the zone
    date = None
    zone = None

    for i, row in df.iterrows():    
        # Extracting Values
        rowValues = list(row)
        
        # Testing for the last row, if yes we stop
        if all(pd.isnull(i) for i in rowValues):
            break

        # These only get zeroed out when the date/zone changes
        # This is also detecting rows that contain totals, we aren't recording these as they can be easily calculated 
        # using the data set
        if customerExpression.match(str(rowValues[0])):
            date = None
            zone = None
            continue

        # Extracting the current date
        if isinstance(rowValues[0], datetime.datetime):
            date = rowValues[0]     


        if zone == None:
            zoneValues = [ word for i, word in enumerate(list(df.iloc[i,:])) if zoneExpression.match(str(word))]
            if len(zoneValues) > 0:
                zone = zoneValues[0]


        driver = rowValues[1]
        truckNumber = rowValues[2]
        garbageTips = rowValues[3]
        garbageWeight = rowValues[4]
        garbageTime = rowValues[5]
        recyclingTips = rowValues[6]
        recyclingWeight = rowValues[7]
        recyclingTime = rowValues[8]
        greenTips = rowValues[9]
        greenWeight = rowValues[10]
        greenTime = rowValues[11]

        # Just skipping some useless rows here (its the greyed rows stuff in the excel files)
        if zone == None:
            continue

        dailyInfo = dailyInfo.append({'fileName':fileName,
                                      'date':date,
                                      'zone':zone,
                                      'driver':driver,
                                      'truckNumber':truckNumber,
                                      'garbageTips':garbageTips,
                                      'garbageWeight':garbageWeight,
                                      'garbageTime':garbageTime,
                                      'recyclingTips':recyclingTips,
                                      'recyclingWeight':recyclingWeight,
                                      'recyclingTime':recyclingTime,
                                      'greenTips':greenTips,
                                      'greenWeight':greenWeight,
                                      'greenTime':greenTime},ignore_index=True)


        # Zeroing out values to prevent bad data
        if customerExpression.match(str(rowValues[0])):
            date = None
            zone = None

In [192]:
dailyInfo.to_csv('C:/Users/Aidan/OneDrive - Simon Fraser University (1sfu)/Garbage Route Optimization/Weekly Weight and Tips/dailyinfo.csv')